In [1]:
import gurobipy as gp
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from scipy.stats import multivariate_normal
from tqdm import tqdm
import matplotlib.pyplot as plt
from dataclasses import dataclass
import os

In [2]:
@dataclass
class Bus:
    def __init__(self, bus_id, pd, gs, gens):
        self.bus_id = bus_id
        self.pd = pd
        self.gs = gs
        self.gens = gens
    def __repr__(self):
        return (f"Bus(pd={self.pd}, gs={self.gs}, gens={self.gens})")
        
@dataclass
class Gen:
    def __init__(self, bus, gen_id, pmin, pmax, pstart, cost):
        self.bus = bus
        self.gen_id = gen_id
        self.pmin = pmin
        self.pmax = pmax
        self.pstart = pstart
        self.cost = cost
    def __repr__(self):
        return (f"Generator(bus={self.bus}, pmin={self.pmin}, pmax={self.pmax}, "
                f"pstart={self.pstart}, cost={self.cost})")

@dataclass
class Line:
    def __init__(self, line_id, rate, frombus, tobus, one_over_reactance):
        self.line_id = line_id
        self.rate = rate
        self.frombus = frombus
        self.tobus = tobus
        self.one_over_reactance = one_over_reactance # β beta
    def __repr__(self):
        return (f"TransmissionLine(rate={self.rate}, frombus={self.frombus}, "
                f"tobus={self.tobus}, one_over_reactance={self.one_over_reactance})")
@dataclass
class NetworkReference:
    def __init__(self, ref, bus, bus_indices, gen, gen_indices, line, line_indices, B, pi, stdomega):
        self.ref = ref # Dictionary of ref data
        self.bus = bus # list of bus
        self.bus_indices = bus_indices
        self.gen = gen
        self.gen_indices = gen_indices
        self.line = line 
        self.line_indices = line_indices
        self.B = B  # Admittance matrix
        self.pi = pi  # π Inverse reduced admittance matrix
        self.stdomega = stdomega  # stdω List of standard deviations
    
@dataclass
class OPFScenarios:
    def __init__(self,noptimal, ref, scenarios, solutions, cbases, rbases, whichbasis, whichscenario, nbases,rbases_all):
        self.noptimal = noptimal
        self.ref = ref
        self.scenarios = scenarios
        self.solutions = solutions
        self.cbases = cbases
        self.rbases = rbases
        self.whichbasis = whichbasis
        self.whichscenario = whichscenario
        self.nbases = nbases
        self.rbases_all = rbases_all

@dataclass
class SingleScenarioOPF:
    def __init__(self, model, p, omega):
        self.model = model
        self.p = p
        self.omega = omega

@dataclass
class Basis_Recourse:
    def __init__(self, ngen, fixed, varying, basiscol, linearterms, constant):
        self.ngen = ngen
        self.fixed = fixed
        self.varying = varying
        self.basiscol = basiscol
        self.linearterms = linearterms
        self.constant = constant

@dataclass
class EnsembleRecourse:
    def __init__(self, ref, baseline, recoursef):
        self.ref = ref
        self.baseline = baseline
        self.recoursef = recoursef

In [3]:
def networkreference(data_file, sigma_scaling=0.05):
    ref = pd.read_excel(data_file, sheet_name=['baseMVA', 'bus','gen','gencost','branch'])
    ref['baseMVA'] = ref['baseMVA'][0].values[0]
    ref['gen'] =  ref['gen'][ref['gen']['status'] == 1]
    ref['gencost'] = ref['gencost'][ref['gencost']["gen_ID"].isin(ref['gen']["gen_ID"].tolist())]
    
    bus_gens = ref["gen"].groupby("bus_i")["gen_ID"].unique().to_dict()
    
    for b in ref["bus"]["bus_i"]:
        if b not in bus_gens:
            bus_gens[b]=[]
            
    scale_factors = np.array([ref['baseMVA']**2, ref['baseMVA'], 1])
    
    gen = {ref['gen'].loc[i,"gen_ID"]: Gen(
                ref['gen'].loc[i,"bus_i"],
                ref['gen'].loc[i,"gen_ID"],
                ref['gen'].loc[i,"Pmin"]/ref['baseMVA'],
                ref['gen'].loc[i,"Pmax"]/ref['baseMVA'],
                ref['gen'].loc[i,"Pg"]/ref['baseMVA'],
                np.array(ref['gencost'].loc[i,['c2','c1','c0']])*scale_factors
            ) for i in ref['gen'].index}
          
    
    bus = {ref['bus'].loc[i,"bus_i"]: Bus(
            ref['bus'].loc[i,"bus_i"],
            ref['bus'].loc[i,"Pd"]/ref['baseMVA'],
            ref['bus'].loc[i,"Gs"]/ref['baseMVA'],
            bus_gens[ref['bus'].loc[i,"bus_i"]],
            ) for i in ref['bus'].index}
          
    
    line = {ref['branch'].loc[l,"line_ID"]: Line(
            l+1, 
            ref['branch'].loc[l,"rateA"]/ref['baseMVA'],
            ref['branch'].loc[l,"bus_i"],
            ref['branch'].loc[l,"bus_j"],
            1/ ref['branch'].loc[l,"x"]
            ) for l in ref['branch'].index
            }
           
    
    ref['ref_buses'] = ref['bus'][ref['bus'].type==3]
    
    B = admittancematrix(ref)
    bus_IDs = sorted(ref['bus']["bus_i"].tolist())
    bus_IDs_Index = {b:i for i,b in enumerate(bus_IDs)}
    B_mat = np.zeros((len(bus_IDs),len(bus_IDs)))
    for b1 in bus_IDs:
        for b2 in bus_IDs:
            B_mat[bus_IDs_Index[b1],bus_IDs_Index[b2]] = B[b1,b2]
    B_red = remove_rows_and_columns(B_mat, 
                                    bus_IDs_Index[ref['ref_buses']["bus_i"].values[0]], 
                                    bus_IDs_Index[ref['ref_buses']["bus_i"].values[0]])
    
    pi_mat = np.linalg.inv(B_red)
    
    pi_mat = add_row_and_column(pi_mat, 
                            bus_IDs_Index[ref['ref_buses']["bus_i"].values[0]], 
                            bus_IDs_Index[ref['ref_buses']["bus_i"].values[0]])
    pi = defaultdict(float)
    for b1 in bus_IDs:
        for b2 in bus_IDs:
            pi[b1,b2]=pi_mat[bus_IDs_Index[b1],bus_IDs_Index[b2]] 
    
    stdomega = [sigma_scaling*(ref['bus'].loc[b,"Pd"]/ref['baseMVA']) for b in ref['bus'].index]
    bus_indices = [i for i in ref['bus']['bus_i']]
    gen_indices = [i for i in ref['gen']['gen_ID']]
    line_indices = [i for i in ref['branch']['line_ID']]

    return NetworkReference(ref,bus, bus_indices,gen,gen_indices,line,line_indices,B,pi,stdomega)

def admittancematrix(ref):
    nbus = len(ref['bus'])
    B = defaultdict(float)
    for br in ref['branch'].index:
        f_bus = ref['branch'].loc[br,"bus_i"]
        t_bus = ref['branch'].loc[br,"bus_j"]
        B[f_bus, t_bus] += (-ref['branch'].loc[br,"x"]/(ref['branch'].loc[br,"x"]**2+ref['branch'].loc[br,"r"]**2)) # imaginary part of admittance, x/(x^2+r^2)
        B[t_bus, f_bus] += (-ref['branch'].loc[br,"x"]/(ref['branch'].loc[br,"x"]**2+ref['branch'].loc[br,"r"]**2)) 
        B[f_bus, f_bus] += (ref['branch'].loc[br,"x"]/(ref['branch'].loc[br,"x"]**2+ref['branch'].loc[br,"r"]**2))
        B[t_bus, t_bus] += (ref['branch'].loc[br,"x"]/(ref['branch'].loc[br,"x"]**2+ref['branch'].loc[br,"r"]**2))
    return B

def remove_rows_and_columns(matrix, rows_to_remove, cols_to_remove):
    matrix = np.delete(matrix, rows_to_remove, axis=0)  # Remove rows
    matrix = np.delete(matrix, cols_to_remove, axis=1)  # Remove columns
    return matrix

def add_row_and_column(matrix, row_index, col_index):
    matrix = np.insert(matrix, row_index, 0, axis=0)
    matrix = np.insert(matrix, col_index, 0, axis=1)
    return matrix

def cost(ref,p):
    try:
        return gp.quicksum(ref.gen[g].cost[0]*p[g] + ref.gen[g].cost[1]*p[g] + ref.gen[g].cost[2] for g in ref.gen_indices)
    except IndexError:
        return gp.quicksum(ref.gen[g].cost[0]*p[g-1] + ref.gen[g].cost[1]*p[g-1] + ref.gen[g].cost[2] for g in ref.gen_indices)

In [4]:
def singlescenarioopf(ref):
    model = gp.Model()
    p = model.addVars(ref.gen_indices, vtype=gp.GRB.CONTINUOUS, name='p',
                    lb=[ref.gen[g].pmin for g in ref.gen_indices],
                    ub=[ref.gen[g].pmax for g in ref.gen_indices])
    
    p.start = [ref.gen[g].pstart for g in ref.gen_indices]
    omega = model.addVars(ref.bus_indices, vtype=gp.GRB.CONTINUOUS, name='omega')

    def busvalue(ref, i): 
        return gp.quicksum(p[g] for g in ref.bus[i].gens) + omega[i] - ref.bus[i].pd - ref.bus[i].gs
    
    def theta(ref, busvalue, i): 
        return gp.quicksum(ref.pi[i,j]*busvalue(ref,j) for j in ref.bus_indices)
    
    def lineflow(l): 
        return ref.line[l].one_over_reactance*(
        theta(ref,busvalue,ref.line[l].frombus) - theta(ref,busvalue,ref.line[l].tobus)
        )
    
    model.addConstrs((lineflow(l) <= ref.line[l].rate for l in ref.line_indices), name='c')

    model.addConstrs((lineflow(l) >= -ref.line[l].rate for l in ref.line_indices), name='c')

    model.addConstr(0 == gp.quicksum(gp.quicksum(p[g] for g in ref.bus[b].gens) +omega[b]-ref.bus[b].pd-ref.bus[b].gs for b in ref.bus_indices), name='c')
    
    model.setObjective(cost(ref,p), gp.GRB.MINIMIZE)
    
    return SingleScenarioOPF(model, p, omega)

def opfscenarios_dist(ref, m, nsamples=1000):
    nonzeroindices = [i for i in range(len(ref.stdomega)) if ref.stdomega[i] > 1e-5]
    mean = np.zeros(len(nonzeroindices))
    cov = np.diag(list(map(ref.stdomega.__getitem__, nonzeroindices)))**2
    omega = multivariate_normal.rvs(mean=mean, cov=cov, size=nsamples)
    omega_samples = np.zeros((len(ref.bus), nsamples))
    omega_samples[nonzeroindices] = omega.T if omega.ndim == 2 else omega[:, np.newaxis]
    return opfscenarios(ref, m, omega_samples)

In [ ]:
def opfscenarios(ref, m, omega_samples):
    nsamples = omega_samples.shape[1]
    status = [None] * nsamples
    soln_p = np.zeros((nsamples, len(ref.gen)))
    cbases = {}
    rbases = {}
    noptimal = 0
    nbases = {}
    rbases_all = {}
    for s in tqdm(range(nsamples)):
        for idx, num in enumerate(m.omega):
            m.omega[num].lb = omega_samples[idx][s]
            m.omega[num].ub = omega_samples[idx][s]
        m.model.setParam('OutputFlag', 0) # suppress the output
        # m.model.setParam('FeasibilityTol', 1e-2) # Default value: 1e-6 Minimum value: 1e-9 Maximum value: 1e-2
        m.model.optimize()
        status[s] = m.model.status
        if status[s] == gp.GRB.OPTIMAL:
            soln_p[s,:] = np.array([v.X for v in m.model.getVars() if 'p' in v.VarName])
            noptimal += 1
            cbasis = tuple(m.model.getAttr('Vbasis', m.model.getVars()))
            rbasis = tuple(m.model.getAttr('Cbasis', m.model.getConstrs()))
            cbases[cbasis] = cbases.get(cbasis, [])
            rbases[rbasis] = rbases.get(rbasis, [])
            cbases[cbasis].append(noptimal)
            rbases[rbasis].append(noptimal)
            nbases[s] = len(rbases)
            rbases_all[s] = rbasis
    assert noptimal == sum(1 for stat in status if stat == gp.GRB.OPTIMAL), 'Mismatch in optimal scenario count'
    sample_p = soln_p[np.array(status)==gp.GRB.OPTIMAL,:]
    sample_omega = omega_samples[:, np.array(status)==gp.GRB.OPTIMAL]
    colbases = list(cbases.keys())
    rowbases = list(rbases.keys())
    whichcol = dict(zip(colbases, range(len(colbases))))
    whichrow = dict(zip(rowbases, range(len(rowbases))))
    whichbasis = np.zeros((noptimal, 2), dtype=int)
    for ckey in cbases.keys():
        whichbasis[cbases.get(ckey)[-1]-1,0] = whichcol[ckey]
    for rkey in rbases.keys():
        whichbasis[rbases.get(rkey)[-1]-1,1] = whichrow[rkey]
    whichscenario = {}
    for i in range(noptimal):
        basiskey = (whichbasis[i,0], whichbasis[i,1])
        whichscenario[basiskey] = whichscenario.get(basiskey, [])
        whichscenario[basiskey].append(i)
    return OPFScenarios(noptimal, ref, sample_omega, sample_p, colbases, rowbases, whichbasis, whichscenario, nbases, rbases_all)

def get_opf_solution(m, omega_samples):
    for idx, num in enumerate(m.omega):
        m.omega[num].lb = omega_samples[idx]
        m.omega[num].ub = omega_samples[idx]
    m.model.optimize()
    assert m.model.status == gp.GRB.OPTIMAL, 'bismillah'
    return np.array([v.X for v in m.model.getVars() if 'p' in v.VarName])

In [6]:
def BasisRecourse(ref, m, cbasis, rbasis):
    br = Basis_Recourse(len(ref.gen), {}, [], {}, np.array([]), [])
    basic_indices = []
    for i, _ in enumerate(ref.gen_indices):
        if cbasis[i] == 0: #basic :Basic
            basic_indices.append(i)
        elif cbasis[i] == -1: #non-basic at lower bound elseif cbasis[i] == :NonbasicAtLower || (cbasis[i] == :Fixed ??)
            br.fixed[i] = m.model.getVars()[i].LB
        elif cbasis[i] == -2: #non-basic at upper bound :NonbasicAtUpper
            br.fixed[i] = m.model.getVars()[i].UB
        else:
            raise ValueError("Unrecognised basis status: {} at index {}".format(scenarios.cbases[0][i], i))
    br.varying = basic_indices
    count = 0
    numbasic = sum(count+1 for i in rbasis if i != 0) #non-basic numbasic = sum(rbasis .!== :Basic)
    basiscol = br.basiscol = dict(zip(basic_indices, range(numbasic)))
    assert len(basic_indices) == numbasic, "Mismatch: len(basic_indices) != numbasic"
    assert basic_indices == sorted(basic_indices), "basic_indices is not sorted"
    assert len(m.model.getA().toarray()) == 2*len(ref.line)+1, 'bismillah'
    basis = np.zeros((numbasic, numbasic), dtype=float)
    omega_matrix = np.zeros((numbasic, len(ref.bus)), dtype=float)
    c = 0
    for i in range(2*len(ref.line)+1): #     for i in 1:(2*ref.nline + 1)
        terms = m.model.getA().toarray()[i] #        terms = m.model.linconstr[i].terms
        if rbasis[i] != 0: # in gurobipy, possible value only 0 and -1; if rbasis[i] !== :Basic
            c += 1
            rhs = m.model.RHS[i] #  rhs = if rbasis[i] == :NonbasicAtLower || rbasis[i] == :Fixedm.model.linconstr[i].lbelseif rbasis[i] == :NonbasicAtUpperm.model.linconstr[i].ub else
# # # chatgpt: model.RHS[i]) already reflects the complete bound as intended in the model formulation. There is no separate "constant" component returned. no need rhs -= terms.constant
            for num, coeff in enumerate(terms): # for (v,coeff) in zip(terms.vars, terms.coeffs)
                if num+1 <= len(ref.gen): # first len(ref.gen) terms are decision variable p                 if v.col <= ref.ngen # power generator
                    if cbasis[num] == 0: # if cbasis[v.col] == :Basic
                        basis[c-1, basiscol[num]] += coeff # basis[c,basiscol[v.col]] += coeff
                    else:
                        rhs -= coeff*br.fixed[num] # rhs -= coeff*br.fixed[v.col]
                else:
                    assert num <= len(ref.gen) + len(ref.bus) # @assert v.col <= ref.ngen + ref.nbus # uncertainty
                    assert cbasis[num] != 0 # @assert cbasis[v.col] !== :Basic # they are fixed for each scenario
                    omega_matrix[c-1, num-len(ref.gen)] -= coeff # ωmatrix[c,v.col-ref.ngen] -= coeff
            br.constant.append(rhs) # push!(br.constant, rhs)
    assert c == numbasic == len(br.varying) == len(br.constant), "bismillah"
    br.linearterms = np.linalg.inv(basis)@omega_matrix
    br.constant = np.linalg.inv(basis)@br.constant
    return br
def get_opf_solution_br(br,omega):
    basic_values = br.linearterms@omega + br.constant
    soln = np.zeros(br.ngen, dtype=float)
    for i in br.fixed.keys():
        soln[i] = br.fixed[i]
    for i in br.varying:
        soln[i] = basic_values[br.basiscol[i]]
    return soln
def theta(ref, busvalue, i):
    return gp.quicksum(ref.pi[i,j]*busvalue(ref,j) for j in ref.bus_indices)
def lineflow(ref, p, omega, l):
    def busvalue(ref, i):
        result = omega[i] - ref.bus[i].pd - ref.bus[i].gs
        if ref.bus[i].gens:
            result += sum(p[g-1] for g in ref.bus[i].gens)
        return result
    return ref.line[l].one_over_reactance*(theta(ref,busvalue,ref.line[l].frombus-1) - theta(ref,busvalue,ref.line[l].tobus-1))
def nviolations(ref, p, omega, atol=1e-5):
    return ngenerationviolations(ref, p) + ntransmissionviolations(ref, p, omega)
def ngenerationviolations(ref, p, atol=1e-5):
    try:
        return sum(ref.gen[i].pmin - atol > p[i] for i in ref.gen_indices)+\
           sum(ref.gen[i].pmax + atol < p[i] for i in ref.gen_indices)
    except IndexError:
        return sum(ref.gen[i].pmin - atol > p[i-1] for i in ref.gen_indices)+\
           sum(ref.gen[i].pmax + atol < p[i-1] for i in ref.gen_indices)
def ntransmissionviolations(ref, p, omega, atol=1e-5):
    bus_idx_map = {k:v for k,v in zip(ref.bus_indices, range(len(ref.bus_indices)))}
    def lineflow(ref, p, omega, l):
        def busvalue(ref, i):
            result = omega[bus_idx_map[i]] - ref.bus[i].pd - ref.bus[i].gs
            if len(ref.bus[i].gens) > 0:
                result += sum(p[g-1] for g in ref.bus[i].gens)
            return result
        return ref.line[l].one_over_reactance*(theta(ref,busvalue,ref.line[l].frombus) - theta(ref,busvalue,ref.line[l].tobus))
    if omega.ndim ==2:
        return sum(ntransmissionviolations_vector(ref, p, omega[i,:], atol=atol) for i in range(omega.shape[0]))
    else:
        return sum(abs(lineflow(ref, p, omega, l).getValue()) > ref.line[l].rate + atol for _, l in enumerate(ref.line_indices))
def ntransmissionviolations_vector(ref, p, omega, atol=1e-5):
    sum(abs(lineflow(ref, p, omega, l).getValue())> ref.line[l].rate + atol for l in ref.line_indices) 
def nviolations(ref, p, omega, atol=1e-5):
    return ngenerationviolations(ref, p) + ntransmissionviolations(ref, p, omega)    
def get_opf_solution_ensemble(er, omega):
    incumbent_p = get_opf_solution_br(er.baseline, omega)
    incumbent_cost = cost(er.ref, incumbent_p)
    feasible_p = nviolations(er.ref, incumbent_p, omega)
    for rf in er.recoursef:
        p = get_opf_solution_br(rf, omega)
        if nviolations(er.ref, p, omega) == 0: # feasible solution
            curr_cost = cost(er.ref, p)
            if not feasible_p:
                incumbent_cost = curr_cost
                incumbent_p = p
                feasible_p = True
            elif curr_cost.getValue() < incumbent_cost.getValue():
                incumbent_cost = curr_cost
                incumbent_p = p
    return incumbent_p

In [7]:
data_file='./excel_outputs/pglib_opf_case240_pserc.xlsx'
nsamples = 5000
ref = networkreference(data_file, sigma_scaling=0.03)
m = singlescenarioopf(ref)
scenarios = opfscenarios_dist(ref, m, nsamples = nsamples)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


100%|██████████| 5000/5000 [00:28<00:00, 172.74it/s]


In [ ]:
br0 = BasisRecourse(ref, m, scenarios.cbases[0], scenarios.rbases[0])
br1 = BasisRecourse(ref, m, scenarios.cbases[1], scenarios.rbases[1])
br2 = BasisRecourse(ref, m, scenarios.cbases[2], scenarios.rbases[2])
br3 = BasisRecourse(ref, m, scenarios.cbases[3], scenarios.rbases[3])
get_opf_solution_br(br0,omega)

ensemble = EnsembleRecourse(ref, br0, [br1,br2,br3])
omega = scenarios.scenarios[:,1]

get_opf_solution_ensemble(ensemble, omega)

In [ ]:
# fig1
# np.random.seed(2)
data_file='./excel_outputs/pglib_opf_case240_pserc.xlsx'
nsamples = 5000
# ref001 = networkreference(data_file, sigma_scaling=0.01)
# ref002 = networkreference(data_file, sigma_scaling=0.02)
ref003 = networkreference(data_file, sigma_scaling=0.03)
# ref004 = networkreference(data_file, sigma_scaling=0.04)
# ref005 = networkreference(data_file, sigma_scaling=0.05)
# m001 = singlescenarioopf(ref001)
# m002 = singlescenarioopf(ref002)
m003 = singlescenarioopf(ref003)
# m004 = singlescenarioopf(ref004)
# m005 = singlescenarioopf(ref005)
# scenarios001 = opfscenarios_dist(ref001, m001, nsamples = nsamples)
# scenarios002 = opfscenarios_dist(ref002, m002, nsamples = nsamples)
scenarios003 = opfscenarios_dist(ref003, m003, nsamples = nsamples)
# scenarios004 = opfscenarios_dist(ref004, m004, nsamples = nsamples)
# scenarios005 = opfscenarios_dist(ref005, m005, nsamples = nsamples)
# m.model.write('gur.lp')
# ref003_test = networkreference(data_file, sigma_scaling=0.03)
# m003_test = singlescenarioopf(ref003_test)
# scenarios003_test = opfscenarios_dist(ref003_test, m003_test, nsamples = nsamples)

In [ ]:
# fig1
# scenarios.nbases
# x1 = list(scenarios001.nbases.keys())
# y1 = list(scenarios001.nbases.values())
# x2 = list(scenarios002.nbases.keys())
# y2 = list(scenarios002.nbases.values())
# x3 = list(scenarios003.nbases.keys())
# y3 = list(scenarios003.nbases.values())
# x4 = list(scenarios004.nbases.keys())
# y4 = list(scenarios004.nbases.values())
# x5 = list(scenarios005.nbases.keys())
# y5 = list(scenarios005.nbases.values())
# plt.figure(figsize=(10, 6))
# plt.plot(x1, y1, marker='o', linestyle='-', color='b', label="σscaling = 0.01")
# plt.plot(x2, y2, marker='s', linestyle='--', color='g', label="σscaling = 0.02")
# plt.plot(x3, y3, marker='^', linestyle='-.', color='r', label="σscaling = 0.03")
# plt.plot(x4, y4, marker='D', linestyle=':', color='c', label="σscaling = 0.04")
# plt.plot(x5, y5, marker='v', linestyle=(0, (5, 1)), color='mb', label="σscaling = 0.05")
# plt.title("Plot of wk Dictionary")
# plt.xlabel("Index")
# plt.ylabel("Value")
# plt.grid(True)
# plt.legend()
# plt.show()

In [ ]:
# table 3
def get_coverage(train_bases, test_bases):
    training_set = set(train_bases.values())
    covered = sum(1 for b in test_bases.values() if b in training_set)
    return covered / len(test_bases)

# coverage_value = get_coverage(scenarios003.rbases_all, scenarios003_test.rbases_all)
# print(f"Cumulative out-of-sample coverage: {coverage_value:.4f}")

sample_sizes = [100, 200, 500, 1000, 2500, 5000]
coverage_results = []
for n in sample_sizes:
    scenarios_train = opfscenarios_dist(ref003, m003, nsamples=n)
    coverage = get_coverage(scenarios_train.rbases_all, scenarios003_test.rbases_all)
    coverage_results.append((n, coverage))
    print(f"Samples: {n}, Out-of-sample coverage: {coverage:.4f}")

In [ ]:
# fig 2
data_file = './excel_outputs/pglib_opf_case300_ieee.xlsx'
def cumulative_coverage_progress(scenarios_train, test_rbases_all):
    discovered = set()
    test_bases = list(test_rbases_all.values())
    progress = []
    
    for i in scenarios_train.rbases_all.keys():
        discovered.add(scenarios_train.rbases_all[i])
        covered = sum(1 for b in test_bases if b in discovered)
        progress.append(covered / len(test_bases))
    return progress

# σ-scaling values to plot
scaling_values = [0.01, 0.02, 0.03, 0.04, 0.05]
colors = ['blue', 'green', 'darkorange', 'purple', 'gold']
labels = [f'σscaling = {s:.2f}' for s in scaling_values]

plt.figure(figsize=(9, 6))

for sigma, color, label in zip(scaling_values, colors, labels):
    ref_train = networkreference(data_file, sigma_scaling=sigma)
    m_train = singlescenarioopf(ref_train)
    scenarios_train = opfscenarios_dist(ref_train, m_train, nsamples=10000)

    # fixed test set for this σ
    ref_test = networkreference(data_file, sigma_scaling=sigma)
    m_test = singlescenarioopf(ref_test)
    scenarios_test = opfscenarios_dist(ref_test, m_test, nsamples=10000)

    coverage_progress = cumulative_coverage_progress(scenarios_train, scenarios_test.rbases_all)

    plt.plot(range(1, len(coverage_progress) + 1), coverage_progress, label=label, color=color)

plt.xlabel("Number of Training Scenarios")
plt.ylabel("Proportion of Scenarios")
plt.title("Proportion of all 10’000 scenarios covered by the already identified bases")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
def rank_bases_by_frequency(scenarios):
    from collections import Counter
    basis_list = list(scenarios.rbases_all.values())
    counter = Counter(basis_list)
    ranked_bases = [basis for basis, _ in counter.most_common()]
    return ranked_bases

def ensemble_policy_performance(ref, test_scenarios, ranked_bases, max_k_list, verbose=True):
    results = []
    omega_test = test_scenarios.scenarios
    true_bases = list(test_scenarios.rbases_all.values())

    for k in max_k_list:
        ensemble = ranked_bases[:k]
        feasible_count = 0
        optimal_count = 0

        for i in range(test_scenarios.noptimal):
            omega_i = omega_test[:, i]
            true_basis = true_bases[i]
            found_feasible = False
            found_optimal = False

            for b in ensemble:
                br = BasisRecourse(ref, m003, scenarios003.cbases[0], b)  # You can use any cbasis (not used)
                p_candidate = get_opf_solution_br(br, omega_i)

                # Check feasibility
                if nviolations(ref, p_candidate, omega_i) == 0:
                    found_feasible = True
                    if b == true_basis:
                        found_optimal = True
                    break  # Stop after first feasible

            if found_feasible:
                feasible_count += 1
            if found_optimal:
                optimal_count += 1

        prop_feas = feasible_count / test_scenarios.noptimal
        prop_opt = optimal_count / test_scenarios.noptimal
        results.append((k, prop_opt, prop_feas))
        if verbose:
            print(f"#Bases: {k}, Optimal: {prop_opt:.3f}, Feasible: {prop_feas:.3f}")
    return results

In [ ]:
# model.write('gur.lp')
# https://jump.dev/JuMP.jl/v0.18/refmodel.html?highlight=write
# writeLP(m.model,"jul.lp";genericnames=false) - write the model to filename in the LP file format. Set genericnames=false for user-defined variable names.
